<a href="https://colab.research.google.com/github/Muhammad-javed440/Q-3/blob/master/tool_calling2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# install pakages
!pip install --upgrade -q langchain-google-genai

In [46]:
from google.colab import userdata
# GEMINI API KEY
GOOGLE_API_KEY= userdata.get("GEMINI_API_KEY")
# WETAHTER API KEY
WEATHER_API_KEY= userdata.get("WATHER_API_KEY")

In [47]:
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display, Markdown

In [48]:
# configure model and
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    api_key=GOOGLE_API_KEY # GEMINI API KEY
)

In [49]:
response=llm.invoke("explain me summary of about red rose in 10 words")
display(Markdown(response.content))

Love, passion, beauty, romance, often symbolizes deep affection.

In [51]:
from langchain_core.tools import tool
import requests

# decorator
@tool
def annual_return(a:int):
  """Returns annual return of a number"""
  return (a +15)/5

# decorator
@tool
def add_two_numbers(a:int, b :int):
  """Returns additional return of a number""" # this doc string
  return (a +b)


# decorator
@tool
def multi(a:int, b:int):
    """Returns multiply return of a number"""
    return( a*b)

@tool
def get_weather_data(city: str):
    """
    Fetches weather data for the given city using OpenWeatherMap API.

    Args:
        city (str): The name of the city to get the weather for.
        api_key (str): Your OpenWeatherMap API key.

    Returns:
        dict: A dictionary containing weather data for the city.
    """
    base_url = f"https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": WEATHER_API_KEY,
        "units": "metric"  # Use "imperial" for Fahrenheit
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        weather_data = response.json()
        return weather_data
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}



In [52]:
tools = [ annual_return, multi , add_two_numbers,get_weather_data]

In [53]:
response = llm.invoke("Whay is the annual return of 10?")
display(Markdown(response.content))

The phrase "annual return of 10" is ambiguous and needs more context. It could mean several things, depending on what you're referring to. Here are the most common interpretations and how to calculate them:

**1. Percentage Return (Most Likely Interpretation):**

*   **Meaning:**  This usually means a 10% annual return. This is the most common way to express investment returns.
*   **Calculation:** A 10% return means that for every $100 you invest, you earn $10 in one year.
    *   **Example:** If you invest $1000 and have a 10% annual return, you will have $1100 at the end of the year ($1000 + $100).

**2. Absolute Return (Less Likely):**

*   **Meaning:** This could mean that your investment grew by 10 units of whatever currency you invested.
*   **Calculation:** If you invested $100 and have an absolute return of 10, you'd end up with $110.
    *   **Example:** If you invested $100 and had an absolute return of 10, you'd end up with $110. The return is simply the difference between the final value and initial value.
*   **Why it's less likely:** This is less common because it's hard to compare across different investments. A return of 10 on a $100 investment is very different than a return of 10 on a $1000 investment.

**3. Return in a Specific Unit (Very Unlikely):**

*   **Meaning:**  In very specific and unusual cases, "return of 10" could refer to a return in some other unit, like bushels of wheat, etc.
*   **Why it's unlikely:** This is almost never the case when discussing standard investments.

**How to Determine the Correct Meaning**

To understand what "annual return of 10" means in your specific case, consider the following:

*   **Context:** Where did you hear or read this phrase? Was it in the context of stocks, bonds, real estate, or something else?
*   **Percentage Sign:** Was there a percentage sign (%) included? If so, it almost certainly means a 10% return.
*   **Specific Units:** Were any specific units mentioned? For example, if someone said, "The farm had an annual return of 10 bushels per acre," that would be a different kind of return.

**In summary, unless there's other information, a phrase like "annual return of 10" almost always refers to a 10% annual return.**

**If you can provide more context, I can give you a more precise answer.** For example, you could tell me:

*   What are you investing in?
*   Where did you encounter this phrase?
*   Are you looking at specific numbers or a general concept?

Knowing this will help me give you a much more accurate answer.

In [54]:

llm_with_tools = llm.bind_tools(tools)

In [55]:
response = llm_with_tools.invoke("What is the anual return  of 20?")

display(Markdown(response.content))

In [56]:
from langchain_core.messages import HumanMessage
query = "Q1 What is the weather of murree ?, Q2 add two number 5 and 6?"

message = [HumanMessage(query)]

display(message)

[HumanMessage(content='Q1 What is the weather of murree ?, Q2 add two number 5 and 6?', additional_kwargs={}, response_metadata={})]

In [57]:
ai_msg = llm_with_tools.invoke(message)
message.append(tools)
message

[HumanMessage(content='Q1 What is the weather of murree ?, Q2 add two number 5 and 6?', additional_kwargs={}, response_metadata={}),
 [StructuredTool(name='annual_return', description='Returns annual return of a number', args_schema=<class 'langchain_core.utils.pydantic.annual_return'>, func=<function annual_return at 0x7d3a548532e0>),
  StructuredTool(name='multi', description='Returns multiply return of a number', args_schema=<class 'langchain_core.utils.pydantic.multi'>, func=<function multi at 0x7d3a549ed360>),
  StructuredTool(name='add_two_numbers', description='Returns additional return of a number', args_schema=<class 'langchain_core.utils.pydantic.add_two_numbers'>, func=<function add_two_numbers at 0x7d3a549ed3f0>),
  StructuredTool(name='get_weather_data', description='Fetches weather data for the given city using OpenWeatherMap API.\n\nArgs:\n    city (str): The name of the city to get the weather for.\n    api_key (str): Your OpenWeatherMap API key.\n\nReturns:\n    dict: 

In [58]:
ai_msg.tool_calls

[{'name': 'get_weather_data',
  'args': {'city': 'murree'},
  'id': 'd76f974f-2538-4a66-97ec-b8282233b2a6',
  'type': 'tool_call'},
 {'name': 'add_two_numbers',
  'args': {'a': 5.0, 'b': 6.0},
  'id': 'c273f59c-a7dc-4789-a007-3104ea51b1c8',
  'type': 'tool_call'}]

In [59]:
for tool_call in      ai_msg.tool_calls:
  selected_tool={
      "add_two_numbers":add_two_numbers,
      "multiply":multi,
      "annual_return":annual_return,
      "get_weather_data":get_weather_data
  }[tool_call["name"].lower()]

  tool_msg = selected_tool.invoke(tool_call)
  # display(tool_msg)
  message.append(tool_msg)
display(message)

[HumanMessage(content='Q1 What is the weather of murree ?, Q2 add two number 5 and 6?', additional_kwargs={}, response_metadata={}),
 [StructuredTool(name='annual_return', description='Returns annual return of a number', args_schema=<class 'langchain_core.utils.pydantic.annual_return'>, func=<function annual_return at 0x7d3a548532e0>),
  StructuredTool(name='multi', description='Returns multiply return of a number', args_schema=<class 'langchain_core.utils.pydantic.multi'>, func=<function multi at 0x7d3a549ed360>),
  StructuredTool(name='add_two_numbers', description='Returns additional return of a number', args_schema=<class 'langchain_core.utils.pydantic.add_two_numbers'>, func=<function add_two_numbers at 0x7d3a549ed3f0>),
  StructuredTool(name='get_weather_data', description='Fetches weather data for the given city using OpenWeatherMap API.\n\nArgs:\n    city (str): The name of the city to get the weather for.\n    api_key (str): Your OpenWeatherMap API key.\n\nReturns:\n    dict: 

In [60]:
ai_msg.tool_calls

[{'name': 'get_weather_data',
  'args': {'city': 'murree'},
  'id': 'd76f974f-2538-4a66-97ec-b8282233b2a6',
  'type': 'tool_call'},
 {'name': 'add_two_numbers',
  'args': {'a': 5.0, 'b': 6.0},
  'id': 'c273f59c-a7dc-4789-a007-3104ea51b1c8',
  'type': 'tool_call'}]